In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Interested Root Codes
root_codes = [11, 17, 12, 13, 16, 15, 14]
root_code_dict = {17: "Coerce",
                  16: "Reduce relation",
                  15: "Exhibit force posture",
                  14: "Protest",
                  13: "Threaten",
                  12: "Reject",
                  11: "Disapprove"}

In [3]:
# Read datasets
bln_nyt = pd.read_csv("PhoenixBLN-NYT_1980-2018.csv")
bln_swb = pd.read_csv("PhoenixBLN-SWB_1979-2019.csv")
fbis = pd.read_csv("PhoenixFBIS_1995-2004.csv")
nyt = pd.read_csv("PhoenixNYT_1945-2005.csv")
wsj = pd.read_csv("PhoenixWSJ_1945-2006.csv")

# Correct bln_nyt column names
bln_nyt.columns = bln_swb.columns

# Add dataset information as column
bln_nyt["dataset"] = "bln_nyt"
bln_swb["dataset"] = "bln_swb"
fbis["dataset"] = "fbis"
nyt["dataset"] = "nyt"
wsj["dataset"] = "wsj"

In [4]:
# Create datasets list
datasets = [bln_nyt, bln_swb, fbis, nyt, wsj]

In [5]:
# Filter datasets based on interested root_codes
bln_nyt = bln_nyt[bln_nyt["root_code"].isin(root_codes)]
bln_swb = bln_swb[bln_swb["root_code"].isin(root_codes)]
fbis = fbis[fbis["root_code"].isin(root_codes)]
nyt = nyt[nyt["root_code"].isin(root_codes)]
wsj = wsj[wsj["root_code"].isin(root_codes)]

In [6]:
print("***Dataset Sizes***")
# Checkout dataset sizes
for a, i in enumerate(datasets):
    print("Dataset:", i["dataset"].values[0], " Shape:", i.shape)

***Dataset Sizes***
Dataset: bln_nyt  Shape: (1416962, 27)
Dataset: bln_swb  Shape: (4255929, 27)
Dataset: fbis  Shape: (817781, 26)
Dataset: nyt  Shape: (1092211, 26)
Dataset: wsj  Shape: (589886, 27)


In [24]:
# Combine all datasets
combined = pd.concat(datasets, axis=0)

# Fix index order
combined.index = range(len(combined))

# Shape of combined dataset
print("Combined dataset size:", combined.shape)

Combined dataset size: (8172769, 27)


In [25]:
# Drop null rows based on source_root column
combined.dropna(subset=["source_root"], inplace=True)

# Shape of combined dataset
print("Combined dataset size after dropping null source_root:", combined.shape)

Combined dataset size after dropping null source_root: (4979422, 27)


In [26]:
# Filter combined dataset based on interested root_codes
combined = combined[combined["root_code"].isin(root_codes)]

# Shape of combined dataset
print("Combined dataset size after filtering interested root_codes:", combined.shape)

Combined dataset size after filtering interested root_codes: (669935, 27)


In [27]:
# Convert root_code values to actual texts
combined["root_code"] = combined["root_code"].map(root_code_dict)

In [28]:
# Column names for duplicate check
required_columns = ["year", "month", "day", "source", "source_root", "target", "target_root", "code", "root_code", "goldstein"]

# Find out duplicate row indexes based on their dataset value counts for each duplicate pair, keep dataset rows with higher value count 
duplicate_check = combined[combined.duplicated(subset=required_columns, keep=False)].sort_values(required_columns)
drop_list = list()
for (_, _, _, _, _, _, _, _, _, _,), group in duplicate_check.groupby(required_columns):
    if group["dataset"].nunique() > 1:
        source_counts = group['dataset'].value_counts()
        most_frequent_source = source_counts.idxmax()
        other_indexes = group[group['dataset'] != most_frequent_source].index.tolist()
        drop_list = other_indexes + drop_list

In [29]:
# Delete unnecessary dataset column
del combined["dataset"]

In [30]:
# Drop duplicate value indexes
combined = combined.drop(index=drop_list)

# Shape of combined dataset
print("Combined dataset size after dropping duplicates:", combined.shape)

Combined dataset size after dropping duplicates: (648311, 26)


In [32]:
# Create RUS source_root
map_source_root = {"FRA": "FRA",
                   "GBR": "GBR",
                   "USA": "USA",
                   "RUS": "RUS",
                   "SUN": "RUS",
                   "POL": "RUS",
                   "CSK": "RUS",
                   "HUN": "RUS",
                   "DDR": "RUS",
                   "ROU": "RUS",
                   "BGR": "RUS"}

# Apply map_source_root
combined["source_root"] = combined["source_root"].replace(map_source_root)

In [34]:
combined = combined[combined['source_root'] != combined['target_root']]

# Shape of combined dataset
print("Combined dataset size after dropping duplicates:", combined.shape)

Combined dataset size after dropping duplicates: (495440, 26)


In [35]:
# Save dataset 
combined.to_csv("combined_final.csv", index=False)

In [36]:
# Split dataset to their periods with filtering
period_1 = combined[(combined['year'] >= 1946) & (combined['year'] <= 1999) & (combined["source_root"].isin(["FRA", "GBR", "USA", "RUS", "SUN", "POL", "CSK", "HUN", "DDR", "ROU", "BGR"]))]
period_2 = combined[(combined['year'] >= 2000) & (combined['year'] <= 2022) & (combined["source_root"].isin(["FRA", "GBR", "USA", "RUS"]))]

In [37]:
# Create reverse map dict based on root_code
reverse_root_code_dict = {v: k for k, v in root_code_dict.items()}

# Convert root_code from text to codes
period_1["root_code"] = period_1["root_code"].map(reverse_root_code_dict)
period_2["root_code"] = period_2["root_code"].map(reverse_root_code_dict)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3512\747304399.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period_1["root_code"] = period_1["root_code"].map(reverse_root_code_dict)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3512\747304399.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period_2["root_code"] = period_2["root_code"].map(reverse_root_code_dict)


In [38]:
# Convert source_root to dataframe index
p1 = period_1.set_index("source_root").sort_values(["source_root", "year", "month", "day"])[["target_root", "target", "root_code", "code", "year", "month", "day"]]
p2 = period_2.set_index("source_root").sort_values(["source_root", "year", "month", "day"])[["target_root", "target", "root_code", "code", "year", "month", "day"]]

In [41]:
p1.to_excel("phoenix list 46-99.xlsx")
p2.to_excel("phoenix list 2000-recent.xlsx")